In [ ]:
# EXTRACCION DE DATOS DE INSTAGRAM 
# DIRECTORIO DEL EJECUTABLE DE CHROME DRIVER
chromedriver_path = '/usr/bin/chromedriver'

In [ ]:
# IMPORTACION DE LAS LIBRERIAS NECESARIAS
from selenium import webdriver
import time
import os
import time
import requests
from pprint import pprint
import pandas as pd
import json
from lxml import html
import re
import csv
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    WebDriverException,
    )
import pandas as pd

from bs4 import BeautifulSoup
import requests

In [ ]:
# ES NECESARIO AGREGR UNA CUENTA DE INSTAGRAM PARA INICIAR SESION
USERNAME = 'USUARIO'
PASSWORD = 'CODIGO'

# TIEMPOS DE ESPERA ENTRE COMANDOS, NO PUEDEN SER DEMASIADO CORTOS PORQUE DENEGA ACCESO A LA CUENTA
WAIT_TIME=8
WAIT_TIME_2=7
WAIT_TIME_3 =6

#NOMBRE DE LOS OBJETOS/PERFILES DE BUSQUEDA
keyword = "heybanco" # /Nu.mx, Bbva, Banorte,...etc


In [ ]:
# INICIO DE SESION
executable_path=os.path.join(chromedriver_path)
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-notifications')
options.add_argument("user-data-dir=/home/klingsor/.config/google-chrome/Default");
options.add_argument("--start-fullscreen")

options = webdriver.ChromeOptions()

preferences = {
        "profile.default_content_setting_values.notifications" : 2,
        "profile.default_content_setting_values.location": 2,
        # We don't need images, only the URLs.
        "profile.managed_default_content_settings.images": 2,
        }

    
options.add_experimental_option("prefs", preferences)
browser = webdriver.Chrome(
        executable_path=executable_path,
        chrome_options=options,
        )

browser.wait = WebDriverWait(browser,WAIT_TIME)   
    
    # ABRE NAVEGADOR
url = "https://www.instagram.com/"
browser.get(url)
    
time.sleep(WAIT_TIME_2)
    
usr = browser.find_element("name", "username")
usr.send_keys(USERNAME)
password = browser.find_element("name","password")
password.send_keys(PASSWORD)
password.send_keys(Keys.RETURN)
time.sleep(WAIT_TIME_2)

# PASE DE VENTANA EMERGENTE
saveinfo = WebDriverWait(browser, WAIT_TIME_3).until(
        EC.element_to_be_clickable((By.XPATH,'//button[contains(text(), "Save info")]'))).click()

time.sleep(WAIT_TIME_2)
 # Get the search box
searchbox = WebDriverWait(browser,WAIT_TIME).until(
EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
    
searchbox.clear()
    
# BUSQUEDA DEL USUARIO/ HASHTAG
searchbox.send_keys(keyword)
time.sleep(WAIT_TIME_3)
searchbox.send_keys(Keys.ARROW_DOWN, Keys.ENTER, Keys.RETURN)
time.sleep(WAIT_TIME_3)


browser.fullscreen_window()
time.sleep(WAIT_TIME)
try:
    image = browser.find_element(By.CLASS_NAME, "_aagu")
   # CLICK EN LA IMAGEN
    image.click()
except Exception as e:
    print("Error:", e)


time.sleep(WAIT_TIME_3)

likes=[]
text_contents=[]
cond=0
# EXTRACCION DE LIKES Y COMENTARIOS
for x in range(200): # NUMERO DE POST DE EXTRACCION
    try:
        link_text = browser.find_element(By.PARTIAL_LINK_TEXT,"likes").text
        likes.append(link_text)
    except Exception as e:  
        print(f"ERROR - Could not fetch likes {e}") 
    try:
        items = browser.find_elements(By.CLASS_NAME, '_ap3a._aaco._aacu._aacx._aad7._aade')        
        for item in items:
            if cond == 0:
                cond=1
            else:
                text_contents.append(item.text)
        cond=0
    except Exception as e:  
        print(f"ERROR - Could not fetch comment  {e}") 
    element = browser.find_element(By.TAG_NAME,'body')  
# PASA AL SIGUIENTE POST
    element.send_keys(Keys.ARROW_RIGHT)
    time.sleep(WAIT_TIME)

browser.quit()

In [ ]:
# EXPORTACION DE DATOS EXTRAIDOS A CSV
import pandas as pd
df_hey = pd.DataFrame(text_contents)
df_hey.to_csv("comentarios-hey-02.csv")